# imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import xgboost as xgb

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# gnn
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv

# autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [4]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [5]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:00,2.703190e+15,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:00,6.304230e+11,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:00,3.885950e+13,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:00,3.534090e+15,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:00,3.755340e+14,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2020-03-10 16:07:00,6.011980e+15,fraud_Fadel Inc,health_fitness,77.00,Haley,Wagner,F,05561 Farrell Crescent,Annapolis,...,39.0305,-76.5515,92106,"Accountant, chartered certified",1943-05-28,45ecd198c65e81e597db22e8d2ef7361,1362931649,38.779464,-76.317042,0
1048571,2020-03-10 16:07:00,4.839040e+15,"fraud_Cremin, Hamill and Reichel",misc_pos,116.94,Meredith,Campbell,F,043 Hanson Turnpike,Hedrick,...,41.1826,-92.3097,1583,Geochemist,1999-06-28,c00ce51c6ebb7657474a77b9e0b51f34,1362931670,41.400318,-92.726724,0
1048572,2020-03-10 16:08:00,5.718440e+11,"fraud_O'Connell, Botsford and Hand",home,21.27,Susan,Mills,F,005 Cody Estates,Louisville,...,38.2507,-85.7476,736284,Engineering geologist,1952-04-02,17c9dc8b2a6449ca2473726346e58e6c,1362931711,37.293339,-84.798122,0
1048573,2020-03-10 16:08:00,4.646850e+18,fraud_Thompson-Gleason,health_fitness,9.52,Julia,Bell,F,576 House Crossroad,West Sayville,...,40.7320,-73.1000,4056,Film/video editor,1990-06-25,5ca650881b48a6a38754f841c23b77ab,1362931718,39.773077,-72.213209,0


## 데이터정리

In [6]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

(214520, 22)

In [7]:
df50 = down_sample_textbook(df02)
df50.shape

(12012, 22)

In [8]:
df50 = df50.reset_index()

In [9]:
N = len(df50)

In [10]:
df50 = df50[["amt","is_fraud"]]

In [11]:
df50["amt"].mean()

297.4638911088911

In [12]:
df50["amt"].describe()

count    12012.000000
mean       297.463891
std        384.130842
min          1.010000
25%         19.917500
50%         84.680000
75%        468.295000
max      12025.300000
Name: amt, dtype: float64

------------------------------------------------------------------------

### tr/test

In [13]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

In [14]:
df50_tr.shape, df50_test.shape

((9009, 2), (3003, 2))

In [15]:
train_mask = [i in df50_tr.index for i in range(N)]
test_mask = [i in df50_test.index for i in range(N)]

In [16]:
train_mask = np.array(train_mask)
test_mask = np.array(test_mask)

In [17]:
train_mask.sum(), test_mask.sum()

(9009, 3003)

In [18]:
train_mask.shape, test_mask.shape

((12012,), (12012,))

------------------------------------------------------------------------

### edge_index 설정

In [17]:
# groups = df50.groupby('cc_num')
# edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
# edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
# edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
# np.save('edge_index_list_plus50.npy', edge_index_list_plus_nparr)

In [19]:
edge_index = np.load('edge_index_list_plus50.npy')
theta = edge_index[:,2].mean()
edge_index = np.load('edge_index_list_plus50.npy').astype(np.float64)
edge_index[:,2] = (np.exp(-edge_index[:,2]/theta) != 1)*(np.exp(-edge_index[:,2]/theta))
edge_index = edge_index.tolist()
mean_ = np.array(edge_index)[:,2].mean()
selected_edges = [(int(row[0]), int(row[1])) for row in edge_index if row[2] > mean_]
edge_index_selected = torch.tensor(selected_edges, dtype=torch.long).t()

------------------------------------------------------------------------

### data설정(x, edge_index, y)

In [20]:
x = torch.tensor(df50['amt'], dtype=torch.float).reshape(-1,1)
y = torch.tensor(df50['is_fraud'],dtype=torch.int64)
data = torch_geometric.data.Data(x=x, edge_index = edge_index_selected, y=y, train_mask = train_mask, test_mask = test_mask)
data

Data(x=[12012, 1], edge_index=[2, 93730], y=[12012], train_mask=[12012], test_mask=[12012])

---

# autogluon

## A. 데이터

In [21]:
tr = TabularDataset(df50_tr)
tst = TabularDataset(df50_test)

## B. predictor 생성

In [22]:
predictr = TabularPredictor("is_fraud")

No path specified. Models will be saved in: "AutogluonModels/ag-20230930_045601/"


## C.적합(fit)

In [23]:
predictr.fit(tr, presets='best_quality')

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230930_045601/"
AutoGluon Version:  0.8.2
Python Version:     3.8.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   749.06 GB / 982.82 GB (76.2%)
Train Data Rows:    9009
Train Data Columns: 1
Label Column: is_fraud
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the

In [24]:
predictr.leaderboard()

                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L2   0.894772       0.019776   4.481569                0.009500           2.139285            2       True         14
1           CatBoost_BAG_L1   0.894661       0.004185   1.490714                0.004185           1.490714            1       True          7
2            XGBoost_BAG_L1   0.894439       0.021677   0.498835                0.021677           0.498835            1       True         11
3      LightGBMLarge_BAG_L1   0.894106       0.006091   0.851570                0.006091           0.851570            1       True         13
4           LightGBM_BAG_L1   0.893995       0.013288   0.571850                0.013288           0.571850            1       True          4
5     NeuralNetTorch_BAG_L1   0.888778       0.050316  14.255491                0.050316          14.255491            1       True         12

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.894772,0.019776,4.481569,0.009500,2.139285,2,True,14
1,CatBoost_BAG_L1,0.894661,0.004185,1.490714,0.004185,1.490714,1,True,7
2,XGBoost_BAG_L1,0.894439,0.021677,0.498835,0.021677,0.498835,1,True,11
3,LightGBMLarge_BAG_L1,0.894106,0.006091,0.851570,0.006091,0.851570,1,True,13
4,LightGBM_BAG_L1,0.893995,0.013288,0.571850,0.013288,0.571850,1,True,4
5,NeuralNetTorch_BAG_L1,0.888778,0.050316,14.255491,0.050316,14.255491,1,True,12
6,LightGBMXT_BAG_L1,0.885004,0.034434,0.492139,0.034434,0.492139,1,True,3
7,KNeighborsUnif_BAG_L1,0.878233,0.010086,0.003595,0.010086,0.003595,1,True,1
8,NeuralNetFastAI_BAG_L1,0.867022,0.086901,7.465914,0.086901,7.465914,1,True,10
9,KNeighborsDist_BAG_L1,0.864136,0.008126,0.002526,0.008126,0.002526,1,True,2


## D. 예측(predict)

In [25]:
(tr.is_fraud == predictr.predict(tr)).mean()

0.8967698967698968

In [26]:
(tst.is_fraud == predictr.predict(tst)).mean()

0.8877788877788878

> 뭐지 best 옵션을 줬는데 더 낮아졌다.